### Introducción

En la clase anterior, aprendimos qué son las APIs y por qué son esenciales. Ahora, daremos un paso práctico y aprenderemos a construir nuestras propias APIs REST utilizando **Flask**, un popular microframework de Python.

**¿Qué es Flask?**
Flask es un "micro" framework web para Python. Se llama "micro" no porque le falten funcionalidades, sino porque su núcleo es simple y extensible. Permite construir aplicaciones web (incluyendo APIs) de manera rápida y con mucha flexibilidad, ya que no impone una estructura de proyecto rígida ni dependencias específicas.

**¿Por qué usar Flask para APIs?**
* **Ligero y Rápido:** Ideal para prototipos y servicios pequeños/medianos.
* **Flexible:** Te da control total sobre los componentes que usas.
* **Fácil de Aprender:** Su sintaxis es intuitiva.
* **Gran Comunidad y Extensiones:** Muchas librerías disponibles para añadir funcionalidades (autenticación, ORMs, etc.).

**Repaso Rápido: API REST**
Una API REST (Representational State Transfer) es un estilo arquitectónico para diseñar aplicaciones en red. Utiliza los métodos HTTP estándar (GET, POST, PUT, DELETE, etc.) para interactuar con recursos, que son identificados por URLs. Las respuestas suelen ser en formato JSON.

### 1. Prerrequisitos y Configuración

Asegúrate de tener Python instalado. Flask se instala fácilmente usando pip.

In [ ]:
# En tu terminal o Anaconda Prompt, ejecuta:
# pip install Flask

Para este notebook, escribiremos el código Flask como si fuera un archivo `.py`. Para ejecutarlo, normalmente guardarías el código en un archivo (ej. `app.py`) y lo ejecutarías desde la terminal (`python app.py`). Dentro de un notebook, podemos simular esto o explicar cómo hacerlo.

---

## 2. Tu Primera Aplicación Flask: "Hola, Mundo API"

Vamos a crear la aplicación Flask más simple posible que devuelva un mensaje JSON.

In [ ]:
# Contenido que iría en un archivo, por ejemplo, 'hello_api.py'

from flask import Flask, jsonify

app = Flask(__name__) # Crea una instancia de la aplicación Flask

@app.route('/', methods=['GET']) # Define la ruta para la URL raíz ('/') y el método HTTP GET
def hola_mundo():
    # jsonify convierte un diccionario de Python a una respuesta JSON
    return jsonify({'mensaje': '¡Hola, Mundo desde mi API con Flask!'})

if __name__ == '__main__':
     # Ejecuta la aplicación en el servidor de desarrollo de Flask
     # debug=True activa el modo de depuración (útil para desarrollo, recarga automáticamente en cambios)
    app.run(debug=True)

**Explicación del Código:**
* `from flask import Flask, jsonify`: Importamos la clase `Flask` para crear nuestra aplicación y `jsonify` para enviar respuestas JSON.
* `app = Flask(__name__)`: Creamos una instancia de la aplicación. `__name__` es una variable especial de Python que aquí ayuda a Flask a encontrar recursos.
* `@app.route('/', methods=['GET'])`: Este es un **decorador** que asocia la URL raíz (`/`) con la función `hola_mundo()`. `methods=['GET']` especifica que esta ruta solo responderá a peticiones HTTP GET.
* `def hola_mundo(): ... return jsonify(...)`: Esta es la **función manejadora** de la ruta. Lo que devuelve esta función es lo que se enviará al cliente. `jsonify` serializa el diccionario a formato JSON y establece el `Content-Type` de la cabecera HTTP a `application/json`.
* `if __name__ == '__main__': app.run(debug=True)`: Esta línea asegura que el servidor de desarrollo de Flask solo se ejecute cuando el script es ejecutado directamente (no cuando es importado como módulo). `debug=True` es muy útil durante el desarrollo.

**Para ejecutar esto:**
1. Guarda el código de arriba en un archivo llamado `hello_api.py`.
2. Abre tu terminal, navega al directorio donde guardaste el archivo.
3. Ejecuta `python hello_api.py`.
4. Verás un mensaje indicando que el servidor está corriendo (usualmente en `http://127.0.0.1:5000/`).
5. Abre tu navegador o Postman y ve a `http://127.0.0.1:5000/`. Deberías ver el JSON: `{"mensaje": "¡Hola, Mundo desde mi API con Flask!"}`.

---

## 3. Definiendo Endpoints (Rutas) y Métodos HTTP

Los endpoints son las URLs específicas que tu API expone. Cada endpoint puede responder a uno o más métodos HTTP.

```python
 from flask import Flask, jsonify, request # request se usa para acceder a datos de la petición

 app = Flask(__name__)

 @app.route('/recurso', methods=['GET'])
 def obtener_recursos():
     return jsonify({'mensaje': 'Lista de recursos'})

 @app.route('/recurso', methods=['POST'])
 def crear_recurso():
     # datos_recibidos = request.get_json() # Obtener datos JSON del cuerpo de la petición
     # Lógica para crear el recurso con datos_recibidos...
     return jsonify({'mensaje': 'Recurso creado'}), 201 # 201 Created

 @app.route('/recurso/<int:id_recurso>', methods=['GET'])
 def obtener_recurso_especifico(id_recurso):
     # Lógica para buscar el recurso con id_recurso...
     return jsonify({'id': id_recurso, 'data': 'Información del recurso específico'})

 if __name__ == '__main__':
     app.run(debug=True)
```
* **Parámetros de Ruta:** En `@app.route('/recurso/<int:id_recurso>')`, `<int:id_recurso>` define un parámetro variable en la URL que se pasa a la función manejadora. `int:` es un convertidor que asegura que el ID sea un entero.

---

## 4. Manejando Peticiones (Requests)

Flask provee el objeto `request` (del módulo `flask`) para acceder a la información de la petición HTTP entrante.

### 4.1. Parámetros de Consulta (Query Parameters)
Son los que vienen después del `?` en una URL (ej. `/buscar?termino=python&limite=10`).

```python
 from flask import request

 @app.route('/buscar', methods=['GET'])
 def buscar():
     termino_busqueda = request.args.get('termino') # Obtiene el valor del parámetro 'termino'
     limite_resultados = request.args.get('limite', default=5, type=int) # Con valor por defecto y tipo
     if not termino_busqueda:
         return jsonify({'error': 'Parámetro "termino" es requerido'}), 400
     return jsonify({
         'buscando': termino_busqueda, 
         'limite': limite_resultados, 
         'resultados': [f'Resultado para {termino_busqueda} 1', f'Resultado para {termino_busqueda} 2'] # Simulación
     })
```

### 4.2. Cuerpo de la Petición (Request Body - JSON)
Para métodos como `POST` o `PUT`, los datos suelen enviarse en el cuerpo de la petición, comúnmente como JSON.

```python
 @app.route('/usuarios', methods=['POST'])
 def crear_usuario():
     datos_usuario = request.get_json() # Parsea el cuerpo JSON a un diccionario de Python
     if not datos_usuario or 'nombre' not in datos_usuario or 'email' not in datos_usuario:
         return jsonify({'error': 'Faltan datos requeridos (nombre, email)'}), 400
     
     nombre = datos_usuario['nombre']
     email = datos_usuario['email']
     # Lógica para guardar el nuevo usuario...
     print(f"Usuario nuevo recibido: Nombre={nombre}, Email={email}")
     return jsonify({'mensaje': f'Usuario {nombre} creado exitosamente'}), 201
```

---

## 5. Ejemplo Completo: API CRUD para una Lista de Tareas

Vamos a construir una API completa con operaciones CRUD (Crear, Leer, Actualizar, Eliminar) para gestionar una lista de tareas. Usaremos una lista en memoria para almacenar los datos (en una aplicación real, usarías una base de datos).

In [ ]:
# Contenido para un archivo como 'tasks_api.py'

from flask import Flask, jsonify, request

app_tasks = Flask(__name__) 

# Almacenamiento en memoria (simulación de base de datos)
tasks = [
    {'id': 1, 'titulo': 'Comprar leche', 'descripcion': 'Leche entera, 1 litro', 'completada': False},
    {'id': 2, 'titulo': 'Llamar a Juan', 'descripcion': 'Recordarle reunión del viernes', 'completada': True}
 ]
next_task_id = 3 # Para generar IDs únicos

# Endpoint para OBTENER TODAS las tareas (GET)
@app_tasks.route('/tasks', methods=['GET'])
def get_tasks():
    return jsonify({'tasks': tasks})

# Endpoint para OBTENER UNA tarea por su ID (GET)
@app_tasks.route('/tasks/<int:task_id>', methods=['GET'])
def get_task(task_id):
    task = next((task for task in tasks if task['id'] == task_id), None)
    if task:
        return jsonify(task)
    return jsonify({'error': 'Tarea no encontrada'}), 404

# Endpoint para CREAR una nueva tarea (POST)
@app_tasks.route('/tasks', methods=['POST'])
def create_task():
    global next_task_id
    if not request.json or 'titulo' not in request.json:
        return jsonify({'error': 'El título es requerido'}), 400
    
    new_task = {
         'id': next_task_id,
         'titulo': request.json['titulo'],
         'descripcion': request.json.get('descripcion', ""), # .get para campos opcionales
         'completada': False
     }
    tasks.append(new_task)
    next_task_id += 1
    return jsonify(new_task), 201 # 201 Created

# Endpoint para ACTUALIZAR una tarea existente (PUT)
@app_tasks.route('/tasks/<int:task_id>', methods=['PUT'])
def update_task(task_id):
    task = next((task for task in tasks if task['id'] == task_id), None)
    if not task:
        return jsonify({'error': 'Tarea no encontrada'}), 404
    
    if not request.json:
        return jsonify({'error': 'Datos no proporcionados'}), 400
        
    task['titulo'] = request.json.get('titulo', task['titulo'])
    task['descripcion'] = request.json.get('descripcion', task['descripcion'])
    task['completada'] = request.json.get('completada', task['completada'])
    
    return jsonify(task)

# Endpoint para ELIMINAR una tarea (DELETE)
@app_tasks.route('/tasks/<int:task_id>', methods=['DELETE'])
def delete_task(task_id):
    global tasks
    task_a_eliminar = next((task for task in tasks if task['id'] == task_id), None)
    if not task_a_eliminar:
        return jsonify({'error': 'Tarea no encontrada'}), 404
    
    tasks = [task for task in tasks if task['id'] != task_id]
    return jsonify({'mensaje': 'Tarea eliminada exitosamente'}), 200 # O 204 No Content

if __name__ == '__main__':
     #app_tasks.run(debug=True, port=5001) # Usar un puerto diferente si el 5000 está ocupado
     app_tasks.run(debug=True) 

**Para probar esta API:**
1. Guarda el código en `tasks_api.py`.
2. Ejecútalo con `python tasks_api.py`.
3. Usa una herramienta como Postman, Insomnia, o `curl` para enviar peticiones:
    * `GET http://127.0.0.1:5000/tasks`
    * `GET http://127.0.0.1:5000/tasks/1`
    * `POST http://127.0.0.1:5000/tasks` con un cuerpo JSON como: `{"titulo": "Nuevo Título", "descripcion": "Descripción detallada"}`
    * `PUT http://127.0.0.1:5000/tasks/1` con un cuerpo JSON para actualizar.
    * `DELETE http://127.0.0.1:5000/tasks/1`

#### Ejemplo Rápido de Prueba con `requests` (desde otro script o notebook):
```python
import requests
BASE_URL = '[http://127.0.0.1:5000](http://127.0.0.1:5000)'

# Obtener todas las tareas
response = requests.get(f"{BASE_URL}/tasks")
print("GET /tasks:", response.json())

# Crear una nueva tarea
nueva_tarea_data = {'titulo': 'Aprender Flask API', 'descripcion': 'Completar el notebook'}
response = requests.post(f"{BASE_URL}/tasks", json=nueva_tarea_data)
print("\nPOST /tasks:", response.status_code, response.json())
tarea_creada_id = response.json().get('id')

# Obtener la tarea creada
if tarea_creada_id:
    response = requests.get(f"{BASE_URL}/tasks/{tarea_creada_id}")
    print(f"\nGET /tasks/{tarea_creada_id}:", response.json())
```

---

## 6. Buenas Prácticas (Muy Brevemente)

* **Usa Códigos de Estado HTTP Correctos:** Ayudan al cliente a entender el resultado de la petición.
* **Manejo de Errores:** Proporciona mensajes de error claros en formato JSON.
* **Validación de Datos de Entrada:** Nunca confíes en los datos del cliente. Valídalos antes de procesarlos (ej. tipos de datos, campos requeridos). Se pueden usar librerías como Marshmallow o Pydantic.
* **Versionado de API:** Si tu API evoluciona, considera versionarla (ej. `/api/v1/recurso`, `/api/v2/recurso`) para no romper la compatibilidad con clientes antiguos.
* **Seguridad:** Autenticación, autorización, HTTPS, etc. (temas más avanzados).
* **Estructura del Proyecto:** Para APIs más grandes, organiza tu código en módulos y blueprints (una característica de Flask).

---

## 7. Resumen

Flask es una herramienta excelente y accesible para comenzar a construir APIs REST en Python. Hemos visto cómo definir rutas, manejar diferentes métodos HTTP, trabajar con datos JSON en peticiones y respuestas, y construir un ejemplo CRUD básico.

Con estos fundamentos, puedes empezar a crear servicios backend que interactúen con otras aplicaciones o frontends.

## (Opcional) Ejercicio Práctico ✍️

1.  **Extiende la API de Tareas:**
    * Añade un campo `prioridad` (ej. 'alta', 'media', 'baja') a las tareas.
    * Modifica el endpoint `POST` para aceptar este nuevo campo (puede ser opcional).
    * Modifica el endpoint `PUT` para permitir actualizar la prioridad.

2.  **Crea una Nueva API CRUD:**
    * Diseña y construye una API para gestionar una colección de "libros". Cada libro debe tener al menos un `id`, `titulo`, `autor` y `anio_publicacion`.
    * Implementa los endpoints GET (todos), GET (específico), POST, PUT y DELETE para los libros.